<a href="https://colab.research.google.com/github/adyasha95/feature-tracking-opencv-demos/blob/main/shifted_image_flow_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
Lucas–Kanade optical flow on a real image sequence created by synthetic shifts.

By default, downloads OpenCV's sample 'building.jpg' and generates a sequence
by shifting it (dx=+2 px, dy=+1 px per frame), then tracks Shi–Tomasi features.

Usage:
  python shifted_image_flow.py
  python shifted_image_flow.py --image path/to/your.jpg
"""

import argparse
import urllib.request
import numpy as np
import matplotlib.pyplot as plt
import cv2

DEFAULT_URL = "https://raw.githubusercontent.com/opencv/opencv/master/samples/data/building.jpg"

def load_gray_from_path_or_url(path: str | None):
    if path:
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        assert img is not None, f"Failed to read image at: {path}"
        return img
    # Download from URL
    resp = urllib.request.urlopen(DEFAULT_URL).read()
    img_arr = np.asarray(bytearray(resp), dtype=np.uint8)
    img = cv2.imdecode(img_arr, cv2.IMREAD_GRAYSCALE)
    assert img is not None, "Failed to download/decode default image."
    return img

def shift_image(img: np.ndarray, dx: int, dy: int) -> np.ndarray:
    M = np.float32([[1, 0, dx], [0, 1, dy]])
    return cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--image", type=str, default=None, help="Path to a local image (optional)")
    parser.add_argument("--frames", type=int, default=20)
    parser.add_argument("--dx", type=int, default=2)
    parser.add_argument("--dy", type=int, default=1)
    parser.add_argument("--max_corners", type=int, default=250)
    args = parser.parse_args()

    base = load_gray_from_path_or_url(args.image)
    frames = [shift_image(base, i * args.dx, i * args.dy) for i in range(args.frames)]

    # Detect corners on first frame
    p0 = cv2.goodFeaturesToTrack(frames[0], maxCorners=args.max_corners, qualityLevel=0.01, minDistance=8, blockSize=7)
    if p0 is None:
        raise SystemExit("No corners found. Try a more textured image or adjust parameters.")
    p0 = np.float32(p0)

    lk_params = dict(
        winSize=(21, 21),
        maxLevel=3,
        criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01),
    )

    mask = np.zeros((frames[0].shape[0], frames[0].shape[1], 3), dtype=np.uint8)
    prev = frames[0]
    out_img = None

    for i in range(1, len(frames)):
        curr = frames[i]
        p1, st, err = cv2.calcOpticalFlowPyrLK(prev, curr, p0, None, **lk_params)
        if p1 is None:
            break

        good_new = p1[st == 1]
        good_old = p0[st == 1]

        frame_bgr = cv2.cvtColor(curr, cv2.COLOR_GRAY2BGR)
        for (x1, y1), (x0, y0) in zip(good_new, good_old):
            x1, y1, x0, y0 = int(x1), int(y1), int(x0), int(y0)
            mask = cv2.line(mask, (x1, y1), (x0, y0), (0, 255, 0), 1)
            frame_bgr = cv2.circle(frame_bgr, (x1, y1), 2, (0, 0, 255), -1)

        out_img = cv2.add(frame_bgr, mask)
        prev = curr.copy()
        p0 = good_new.reshape(-1, 1, 2)

    if out_img is not None:
        plt.figure(figsize=(8, 6))
        plt.imshow(out_img[..., ::-1])  # BGR->RGB
        plt.title("Optical Flow Tracks (Shifted Real Image)")
        plt.axis("off")
        plt.tight_layout()
        plt.show()

if __name__ == "__main__":
    main()